In [ ]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import warnings
import math
import pandas as pd
import random

warnings.filterwarnings("ignore") # To ignore warnings produced

In [ ]:
zipcodes_path = 'pittsburgh-allegheny-county.csv'
data = np.genfromtxt(zipcodes_path, dtype=str, delimiter=',', encoding='utf-8-sig')
neighborhoods = data.astype(np.int)

POD_sites_path = 'POD Sites.xlsx'
POD_df = pd.read_excel(POD_sites_path)
POD_df = POD_df[['SCHOOL/FACILITY NAME', 'STRIP MAP']]
POD_df['ZIPCODE'] = POD_df['STRIP MAP'].apply(lambda x: str(x)[-5:])
POD_df = POD_df[:47]
POD_df

POD_zips = np.array(pd.to_numeric(POD_df.ZIPCODE).values)
print(POD_zips)
print(neighborhoods)

In [ ]:
zipDF = pd.read_csv('zipcodes.csv')
zipDF_PA = zipDF.loc[zipDF['state'] == 'PA']

In [ ]:
num_neighborhoods = len(neighborhoods)
num_sites = len(POD_zips)


random.seed(20)

zipcodes_df = pd.read_csv('zip_code_database.csv')
zipcodes_df = zipcodes_df.loc[zipcodes_df['type'].isin(['STANDARD', 'UNIQUE'])]
zipcodes_df = zipcodes_df.loc[zipcodes_df['county'] == 'Allegheny County']
zipcodes_df_filtered = zipcodes_df[['zip', 'primary_city', 'latitude', 'longitude', 'irs_estimated_population']]
zipcodes_df_filtered

from math import sin, cos, sqrt, atan2, radians

def calcDistBetweenTwoPoints(pt1, pt2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(pt1[0])
    lon1 = radians(pt1[1])
    lat2 = radians(pt2[0])
    lon2 = radians(pt2[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return(distance)
    
def getLatLongFromZip_graph(zipcode, df):
    lat = df.loc[df['zip_code'] == zipcode]['latitude'].values[0]
    long = df.loc[df['zip_code'] == zipcode]['longitude'].values[0]
    return((lat, long))

def getLatLongFromZip(zipcode, df):
    lat = df.loc[df['zip'] == zipcode]['latitude'].values[0]
    long = df.loc[df['zip'] == zipcode]['longitude'].values[0]
    return((lat, long))
    
distances = []
for i in range(num_neighborhoods):
    temp = []
    for j in range(num_sites):
        try:
#             print(POD_zips[j])
            pt1 = getLatLongFromZip(neighborhoods[i], zipcodes_df_filtered)
            pt2 = getLatLongFromZip(POD_zips[j], zipcodes_df_filtered)
            dist = calcDistBetweenTwoPoints(pt1, pt2)
            temp.append(dist)
        except:
            temp.append(random.randint(150,200))
    distances.append(temp)
distances = np.array(distances)

In [ ]:
POD_lats = []
POD_longs = []

for z in POD_zips:
    lat, long = getLatLongFromZip_graph(z, zipDF_PA)
    POD_lats.append(lat)
    POD_longs.append(long)
    
neighborhood_lats = []
neighborhood_longs = []

for z in neighborhoods:
    lat, long = getLatLongFromZip_graph(z, zipDF_PA)
    neighborhood_lats.append(lat)
    neighborhood_longs.append(long)

    
plt.scatter(neighborhood_longs, neighborhood_lats, marker='o', c = 'blue')
plt.scatter(POD_longs, POD_lats, marker='x', c = 'red')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Neighborhoods (blue) vs possible POD sites (red)')




In [ ]:
population = []
for i in range(num_neighborhoods):
    try:
        population.append(zipcodes_df_filtered.loc[zipcodes_df_filtered['zip'] == neighborhoods[i]]['irs_estimated_population'].values[0])
    except:
        population.append(0)
population = np.array(population)

for i in range(len(population)):
    population[i] = population[i] + 13000
print(population)

In [ ]:
B = np.array(pd.read_csv("all_scenarios_test.csv", header = None))

In [ ]:
# MODEL INITIALIZATION
m = Model()
sizes = range(3)
treatments = range(2)
zipcodes = range(92)
sites = range(47)
days = range(25)
scenarios = range(B.shape[0]) 

# CONSTANTS
D = distances
p = population
e = np.array([72, 85, 100])
o = 20000
v = 50
f = 8000
r = 206.76
h = 12.5
C = 1370
B = np.array(pd.read_csv("all_scenarios_test.csv", header = None))
K = 332456550

# DECISION VARIABLES
## FIRST STAGE
y = m.addVars(sites, vtype = GRB.BINARY)
T = m.addVars(sites, vtype = GRB.BINARY)
## SECOND STAGE
A = m.addVars(zipcodes, sites, scenarios, vtype = GRB.BINARY)
S = m.addVars(sites, scenarios, vtype = GRB.BINARY)
M = m.addVars(sites, scenarios, vtype = GRB.BINARY)
L = m.addVars(sites, scenarios, vtype = GRB.BINARY)
U = m.addVars(sites, days, scenarios, vtype = GRB.BINARY)
X = m.addVars(sites, days, scenarios)#, vtype = GRB.INTEGER)
I = m.addVars(sites, days, scenarios)#, vtype = GRB.INTEGER)

# OBJECTIVE
z1 = m.addVar(lb = 0.0)
z2 = m.addVar(lb = 0.0)
avg_distance1 = LinExpr()
avg_distance2 = LinExpr()

# CONSTRAINTS
m.addConstr(o*sum(y[j] for j in sites) + (v/len(scenarios))*sum(A[i,j,k]*p[i] for i in zipcodes for j in sites for k in scenarios) + (1/len(scenarios))*sum(r*X[j,t,k] + h*I[j,t,k] + f*U[j,t,k] for j in sites for t in days for k in scenarios) <= K)
for k in scenarios: 
    infected_pop = sum(p[i]*B[i,k] for i in zipcodes)
    uninfected_pop = sum(p[i]*(1-B[i,k]) for i in zipcodes) 
    for i in zipcodes:
        m.addConstr(sum(A[i,j,k] for j in sites) == 1)

        for j in sites:
            m.addConstr(A[i,j,k] <= y[j])
            m.addConstr(A[i,j,k] + (B[i,k] - T[j]) <= 1)
            m.addConstr(A[i,j,k] + (T[j] - B[i,k]) <= 1)
            m.addConstr(A[i,j,k] >= 0)
            avg_distance1 += p[i]*B[i,k]*D[i,j]*A[i,j,k]/infected_pop
            avg_distance2 += p[i]*(1-B[i,k])*D[i,j]*A[i,j,k]/uninfected_pop
            
avg_distance1 /= len(scenarios)

avg_distance2 /= len(scenarios)

m.addConstr(z1 == avg_distance1)
m.addConstr(z2 == avg_distance2)
            
for j in sites:
    m.addConstr(y[j] >= 0)
    m.addConstr(T[j] >= 0)
    for k in scenarios:
        m.addConstr(S[j,k] + M[j,k] + L[j,k] <= y[j])
        m.addConstr(e[0]*sum(X[j,t,k] for t in days) >= sum(A[i,j,k]*p[i] for i in zipcodes) - (1-S[j,k])*1000000)
        m.addConstr(e[1]*sum(X[j,t,k] for t in days) >= sum(A[i,j,k]*p[i] for i in zipcodes) - (1-M[j,k])*1000000)
        m.addConstr(e[2]*sum(X[j,t,k] for t in days) >= sum(A[i,j,k]*p[i] for i in zipcodes))
        m.addConstr(S[j,k] >= 0)
        m.addConstr(M[j,k] >= 0)
        m.addConstr(L[j,k] >= 0)
        
        administered = [0, 0, 0]
        for t in days:
            administered[0] += e[0]*X[j,t,k]
            administered[1] += e[1]*X[j,t,k]
            administered[2] += e[2]*X[j,t,k]
            m.addConstr(X[j,t,k] >= 10*M[j,k] + 20*L[j,k] - 20*(1 - U[j,t,k]))
            m.addConstr(X[j,t,k] <= 10*S[j,k] + 20*M[j,k] + C*L[j,k])
            m.addConstr(X[j,t,k] <= 1000000*U[j,t,k])
            m.addConstr(U[j,t,k] <= S[j,k] + M[j,k] + L[j,k])
            m.addConstr(I[j,t,k] + (1-S[j,k])*1000000 >= sum(A[i,j,k]*p[i] for i in zipcodes) - administered[0])
            m.addConstr(I[j,t,k] + (1-M[j,k])*1000000 >= sum(A[i,j,k]*p[i] for i in zipcodes) - administered[1])
            m.addConstr(I[j,t,k] + (1-L[j,k])*1000000 >= sum(A[i,j,k]*p[i] for i in zipcodes) - administered[2])
            m.addConstr(X[j,t,k] >= 0)
            m.addConstr(I[j,t,k] >= 0)
            m.addConstr(U[j,t,k] >= 0)
            
            
for j in sites:
    for k in scenarios:
        m.addConstr(sum(X[j,t,k] for j in sites) <= C)
        
    
        


In [ ]:
alpha_values = [0, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99, 0.999, 1]
obj_vals = np.zeros([len(alpha_values), 2])
for i, alpha in enumerate(alpha_values):
    # OBJECTIVE
    m.setObjective(alpha * z1 + 
                    (1-alpha) * z2)
    m.modelSense = GRB.MINIMIZE
    m.optimize()
                    
    print("Average Distance for Therapeutics:", avg_distance1.getValue())
    print("Average Distance for Vaccines:", avg_distance2.getValue())
    obj_vals[i,0] = z1.x
    obj_vals[i,1] = z2.x